In [3]:
%%capture
!apt-get install  aria2
!apt-get install lzop


In [4]:
%%capture
import os, json, zipfile, shutil, platform, time

import scipy.sparse as sps
from pandas import DataFrame
import pandas as pd
import numpy as np

import multiprocessing
import multiprocessing.pool
import time

from random import randint
from tensorflow import keras
targetCol="like"
targets=["like",	"retweet",	"retweetCom",	"reply",	"interaction"]
droppable=[t for t in targets if t!=targetCol]
import gc
import numpy as np
np.random.seed(1234)
gc.collect()

#Disable copy warning (hopefully are false positive here)
pd.options.mode.chained_assignment = None

In [6]:
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",\
               "engaging_user_account_creation", "engagee_follows_engager","reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]

#Metrics

In [13]:
from sklearn.metrics import average_precision_score, log_loss
def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred-1e-7)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/(strawman_cross_entropy+1e-7))*100.0

def calculate_metrics(prediction,targetTest):
    rce = compute_rce(prediction,targetTest)
    average_precision = average_precision_score(targetTest,prediction)
    print(f"rce={rce}")
    print(f"AP={average_precision}")

# Preprocessing function

###Special Tokens

In [14]:
HTTPS_token = '14120' 
RT_token = '56898'
at_token = '137'
hashtag_token = '108'
gt_token = '135'
lt_token = '133'
amp_token = '111'
question_token = '136'
esclamation_token = '106'
period_token = '119'
two_periods_token = '131'
coma_token = '117'
dollar_token = '109'
period_coma_token = '132'
parenthesis_open_token = '113'
parenthesis_closed_token = '114'
star_token = '115'
slash_token = '120'
line_token = '118'
underscore_token = '168'
tilde_token = '198'
virgolette_token = '107'
square_parenthesis_open_token = '164'
square_parenthesis_closed_token = '166'
unk_token = '100'
others_tokens = ['11733', '12022']
twitter_internal_link="188" #"t":188,#twitter links
co_domain_token= "11170" #"co":11170
eight="129" #"8":129 ???? forse utile

special_tokens_list = [
    at_token,
    hashtag_token,
    gt_token,
    lt_token,
    amp_token,
    question_token,
    esclamation_token,
    period_token,
    coma_token,
    dollar_token,
    period_coma_token,
    two_periods_token,
    parenthesis_open_token,
    parenthesis_closed_token,
    star_token,
    slash_token,
    line_token,
    underscore_token,
    tilde_token,
    virgolette_token,
    square_parenthesis_open_token,
    square_parenthesis_closed_token,
    unk_token,
    twitter_internal_link, 
    co_domain_token,
    eight, 
]

jimin="27128"
V="159"
BTS="BTS"
btsTokenized="70447\t10731\t"
BE="46291" #album name
bts=[
    jimin,
    V,
    BE,
    BTS
]

EA="38478"
SPORTS="15506"

sport=[
    EA,
    SPORTS,
]

quarterback="154"
ncaa="23864"
FOOTBALL="12499"
nfl="20179"
Denard_Robinson=[11274	,10235]
football=[
        *sport,
        quarterback,
        ncaa,
        FOOTBALL,
        nfl
]
def countList(list_specific):
    def count_f(row):
        row=row.split("\t")
        count=0
        for el in row:
            if el in list_specific:
                count+=1
        return count
    return count_f

countSpecial=countList(special_tokens_list)

countBTS=countList(bts)

countSports=countList(sport)

countFootball=countList(football)


def countSpecificWord(word):
    def count_f(row):
        row=row.split("\t")
        return row.count(word)
    return count_f

def countWordSet(words):
    def count_f(row):
        row=row.split("\t")
        count=0
        for w in words:
            count+=row.count(w)
        return w
    return count_f

def checkBalanced(a,b):
    def count_f(row):
        row=row.split("\t")
        counta=row.count(a)
        countb=row.count(b)
        return 1 if counta==countb else 0
    return count_f


In [15]:
def removeRT(row):
    if row.find("101\t56898")!=-1:
        idx=row.find("131")
        return row[:3]+row[idx+3:]
    return row

###Hashtags

In [16]:
origin="drive/MyDrive/twitter/"
files=["ErdoganHashtags.bz2","BigBrotherHashtags.bz2",
       "BTSHashtags.bz2","NCTHashtags.bz2","GOT7Hashtags.bz2","ThaiHashtags.bz2",
       "BirmaniaHashtags.bz2","CovidHashtags.bz2","ImpactHashtags.bz2","PeingHashtags.bz2"]
res=[]
import json

for file in files:
    os.system(f"cp {origin+file} {file}")
    os.system(f"bunzip2 {file}")
    os.system(f"mv {file[:-4]} {file[:-4]}.json")
    with open(f'{file[:-4]}.json') as f:
        data = json.load(f)
        res.append(set(data.keys()))
        del data
        gc.collect()


countHashtagFunctions=[]
for i in range(len(files)):
    countHashtagFunctions.append([f"count_{i}",countList(res[i])])


###actual code

In [17]:
def countMedia(arr,mediaType):
        c=0
        for m in arr:
            if m==mediaType:
                c+=1
        return c

In [18]:
import re
summary={}
def computeTrainTest(verbose=False,userSpecific=False):
    df=pd.read_csv("train.csv",header=None,sep='\x01')
    df.columns=all_features

    df["like"]=df["like_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
    df.drop("like_timestamp",axis=1,inplace=True)

    df["retweet"]=df["retweet_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
    df.drop("retweet_timestamp",axis=1,inplace=True)

    df["retweetCom"]=df["retweet_with_comment_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
    df.drop("retweet_with_comment_timestamp",axis=1,inplace=True)

    df["reply"]=df["reply_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
    df.drop("reply_timestamp",axis=1,inplace=True)

    df["interaction"]=df["reply"]+df["retweetCom"]+df["retweet"]+df["like"]

    df["interaction"][df["interaction"]>0]=1


    df.fillna(value={"present_media":""},inplace=True)
    df["photos"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"Photo")).astype("uint8")
    df["gifs"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"GIF")).astype("uint8")
    df["videos"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"Video")).astype("uint8")

    #df["NoText"]=df["text_tokens"].apply(lambda row:  False if len(row.split("\t"))>2 else True).astype("uint16")

    df["distinct_text_tokens"]=df["text_tokens"].apply(lambda row: len(set(row.split("\t")))-2 if row!="" else 0).astype("uint16")

    df["special_characters"]=df["text_tokens"].apply(countSpecial).astype("uint16")

    df["text_tokens"]=df["text_tokens"].apply(lambda row:removeRT(row))

    #remove links
    df["text_tokens"]=df["text_tokens"].apply(lambda row: re.sub("14120\t131\t120\t120\t188\t119\t11170\t120\t\d+\t\d+\t\d+\t\d+\t\d+\t\d+\t\d+\t\d+\t","",row) )

    
    
    
    df["text_tokens"]=df["text_tokens"].apply(lambda row: re.sub("70447\t10731\t","BTS",row))

    df["isReplying"]=df["text_tokens"].apply(lambda row: row.startswith("101\t137"))

    df["unknown"]=df["text_tokens"].apply(countSpecificWord(unk_token)).astype("uint16")
    
    #df["sentence"]=df["text_tokens"].apply(countSpecificWord(period_token)).astype("uint16")

    df["esclamations"]=df["text_tokens"].apply(countSpecificWord(esclamation_token)).astype("uint16")

    df["questions"]=df["text_tokens"].apply(countSpecificWord(question_token)).astype("uint16")

    df["mentions"]=df["text_tokens"].apply(countSpecificWord(at_token)).astype("uint16")

    df["twitter_Specific"]=df["text_tokens"].apply(countWordSet([RT_token,at_token,hashtag_token])).astype("uint16")

    #RT token at the beginning is removed so remaining RT tokens should indicate a request for retweet
    df["asking_retwet"]=df["text_tokens"].apply(countWordSet([RT_token])).astype("uint16")

    df["ask_reply"]=df["text_tokens"].apply(lambda row: row.count("76456\t10454"))

    #Noisy the word like has other meanings
    df["ask_like"]=df["text_tokens"].apply(lambda row: row.count("11850"))


    df["subsentence"]=df["text_tokens"].apply(countSpecificWord(coma_token)).astype("uint16")

    df["subsentence_semantic_separation"]=df["text_tokens"].apply(countSpecificWord(period_coma_token)).astype("uint16")

    #df["BTS"]=df["text_tokens"].apply(countBTS).astype("uint16")

    #df["Sports"]=df["text_tokens"].apply(countSports).astype("uint16")

    #df["Football"]=df["text_tokens"].apply(countFootball).astype("uint16")

    #df["proportion"]=df.apply(lambda row: row["distinct_text_tokens"]/max(1,row["special_characters"]))
    

    

    #df["balanced_angular"]=df["text_tokens"].apply(checkBalanced(gt_token,lt_token)).astype("uint16")

    #df["balanced_square"]=df["text_tokens"].apply(checkBalanced(square_parenthesis_open_token,square_parenthesis_closed_token)).astype("uint16")

    #df["balanced_round"]=df["text_tokens"].apply(checkBalanced(parenthesis_open_token,parenthesis_closed_token,)).astype("uint16")

    df["parenthesis"]=df["text_tokens"].apply(countWordSet([parenthesis_open_token,parenthesis_closed_token,gt_token,lt_token,square_parenthesis_open_token,square_parenthesis_closed_token])).astype("uint16")
    
    

    df["text_tokens"]=df["text_tokens"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint16")
    df.fillna(value={"hashtags":""},inplace=True)
    
    

    df["present_media"]=df["present_media"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
    df.fillna(value={"present_links":""},inplace=True)
    df["present_links"]=df["present_links"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
    df.fillna(value={"present_domains":""},inplace=True)
    df["present_domains"]=df["present_domains"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
    
    df=pd.get_dummies(df,columns=["tweet_type"])

    gc.collect()
    df.drop("tweet_id",axis=1,inplace=True)
    df.drop("engaging_user_account_creation",axis=1,inplace=True)
    df.drop("engaged_with_user_account_creation",axis=1,inplace=True)
    df.drop("tweet_timestamp",axis=1,inplace=True)

    df.drop("engaged_with_user_id",axis=1,inplace=True)
    
    df.drop("language",axis=1,inplace=True)
    
    
    #TODO Normalization
    msk = np.random.rand(len(df)) < 0.8
    train=df[msk]
    test=df[~msk]
    
    hashtagCols=[]
    print("here")
    for f in countHashtagFunctions:
        hashtagCols.append(f[0])
        train[f[0]]=train["hashtags"].apply(f[1]).astype("uint8")
        test[f[0]]=test["hashtags"].apply(f[1]).astype("uint8")
        
    if userSpecific:
        tempTable=train[train["interaction"]>0].copy()

        tempTable=tempTable[[*hashtagCols,"engaging_user_id"]]

        tempTable=tempTable.groupby(["engaging_user_id"]).sum()

        trainTable=tempTable.copy()
        testTable=tempTable.copy()
        for c in hashtagCols:
            trainTable[c]=trainTable[c].apply(lambda row: 0 if row<2 else 1)
            testTable[c]=testTable[c].apply(lambda row: 0 if row<1 else 1)
        if verbose:
            print(trainTable[:20])
        train=train.merge(trainTable,how="left",on="engaging_user_id")
        
        test=test.merge(testTable,how="left",on="engaging_user_id")
        

    train.fillna(0,inplace=True)
    test.fillna(0,inplace=True)
    train.drop("engaging_user_id",axis=1,inplace=True)
    test.drop("engaging_user_id",axis=1,inplace=True)

    train["hashtags"]=train["hashtags"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
    test["hashtags"]=test["hashtags"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")

    for col in train.columns:
        if col  in droppable:
            train.drop(col,axis=1,inplace=True)
            test.drop(col,axis=1,inplace=True)
        elif col in ["engaged_with_user_id","engaging_user_id","language"]:
            train.drop(col,axis=1,inplace=True)
            test.drop(col,axis=1,inplace=True)
        else:
            train[col]=train[col].astype("float32")
            test[col]=test[col].astype("float32")
    for col in train.columns:
        if col!=targetCol:
            if col not in summary:
                summary[col]={"mean":np.mean(train[col].values),"std":np.std(train[col])}
            if summary[col]["std"]==0:
                train[col]=0
                test[col]=0
            else:
                train[col]=(train[col]-summary[col]["mean"])/summary[col]["std"]
                test[col]=(test[col]-summary[col]["mean"])/summary[col]["std"]


    target=train[targetCol]
    target=target.astype("float32")
    target=target.values
    targetTest=test[targetCol]
    targetTest=targetTest.astype("float32")
    targetTest=targetTest.values
    train.drop(targetCol,axis=1,inplace=True)
    test.drop(targetCol,axis=1,inplace=True)
    input=train.values
    inputTest=test.values
    del df
    del train
    del test
    return (input,target,inputTest,targetTest)

#Code for model adding features from old trained networks

##NN model

In [19]:
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from math import ceil
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.optimizers import RMSprop
from tensorflow.keras import regularizers

In [20]:
import tensorflow.keras.backend as K
def create_weighted_binary_crossentropy(zero_weight, one_weight):
    def weighted_binary_crossentropy(y_true, y_pred):
        b_ce = K.binary_crossentropy(y_true, y_pred)
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce
        return K.mean(weighted_b_ce)
    return weighted_binary_crossentropy

In [21]:
def buildModel(inputSize):
    model = Sequential()
    size=128
    reduction_factor=2
    shape=(inputSize,)
    depth=2
    dropout=0.4
    kernel_resularizer_norm=1e-5
    for i in range(depth):
                model.add(Dense(size,  input_shape=shape,kernel_regularizer=regularizers.l2(kernel_resularizer_norm)))
                model.add(Activation("relu"))
                model.add(Dropout(dropout))
                size=size//reduction_factor
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(32,kernel_regularizer=regularizers.l2(kernel_resularizer_norm)))
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Dense(1,kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Activation("sigmoid"))
    model2=keras.Model(inputs=[model.inputs],outputs=model.layers[-3].output)

    return (model,model2)

In [22]:
import copy
def computeLast(arr,last):
    return last.predict(arr)

from tqdm import tqdm
def trainModel(train,trainTarget,test,testTarget,last=None):
    if last!=None:
        originTrain=copy.deepcopy(train)
        originTest=copy.deepcopy(test)
        for m in tqdm(last, position=0, leave=True):
            temp=computeLast(train,m)
            train=np.hstack((originTrain,temp))
            temp=computeLast(test,m)
            test=np.hstack((originTest,temp))
            gc.collect()
        del temp
    count=len(train[0])
    print(f"features: {count}")
    model,model2=buildModel(count)
    BS=64
    loss=create_weighted_binary_crossentropy(1,1.1)
    model.compile(loss=loss, metrics=[tf.keras.metrics.AUC(curve="PR",name="PRAUC"),"accuracy"],optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))
    model.fit(train,trainTarget,epochs=1,batch_size=BS)
    prediction=model.predict(test)
    calculate_metrics(prediction,testTarget)
    return (model,model2)

##Chunk wise download and work

In [ ]:
import os 
from tqdm import tqdm
import json

gc.collect()
modelComp=[]
modelParz=[]
index=0
with open("training_urls.txt") as doc:
    index=0
    for line in doc:#tqdm(doc, position=0, leave=True):
        #skip lzo.index files (odd row number)
        if index%2==1:
            index+=1
            continue
        os.system(f"aria2c '{line}'")
        partName=str(index//2)
        print(partName)
        partName="0"*(5-len(partName))+partName
        
        os.system(f'lzop -x part-{partName}.lzo')
        os.system(f'mv part-{partName} train.csv')
        os.system(f"rm part-{partName}.lzo")
        train,trainTarget,test,testTarget=computeTrainTest()
        if len(modelComp)==0:
            model1,model2=trainModel(train,trainTarget,test,testTarget)
        else:
            model1,model2=trainModel(train,trainTarget,test,testTarget,last=modelComp)
        modelComp.append(model1)
        modelParz.append(model2)
        os.system("rm train.csv")
        os.system(f"rm part-{partName}.lzo.aria2")
        os.system(f"rm part-{partName}.lzo.index")
        index+=1
        gc.collect()
os.system("mv ./temp.txt ./dictionary.txt")

#Code for modeling prediction error

##NN model

In [ ]:
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from math import ceil
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.optimizers import RMSprop
from tensorflow.keras import regularizers

In [ ]:
import tensorflow.keras.backend as K
def create_weighted_binary_crossentropy(zero_weight, one_weight):
    def weighted_binary_crossentropy(y_true, y_pred):
        b_ce = K.binary_crossentropy(y_true, y_pred)
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce
        return K.mean(weighted_b_ce)
    return weighted_binary_crossentropy

In [ ]:
def buildModel(inputSize,probabilistic=False):
    model = Sequential()
    size=32
    reduction_factor=2
    shape=(inputSize,)
    depth=1
    dropout=0.5
    kernel_resularizer_norm=1e-5
    for i in range(depth):
                model.add(Dense(size,  input_shape=shape,kernel_regularizer=regularizers.l2(kernel_resularizer_norm)))
                model.add(Activation("relu"))
                model.add(Dropout(dropout))
                size=size//reduction_factor
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(8,kernel_regularizer=regularizers.l2(kernel_resularizer_norm)))
    model.add(Activation("relu"))
    model.add(Dropout(dropout))
    model.add(Dense(1,kernel_regularizer=regularizers.l2(1e-4)))
    if (probabilistic):
        model.add(Activation("sigmoid"))
    else:
        model.add(Activation("tanh"))
    model2=keras.Model(inputs=[model.inputs],outputs=model.layers[-3].output)

    return (model,model2)

In [ ]:
import copy
def computeLast(arr,last):
    return last.predict(arr)

from tqdm import tqdm
def trainModel(train,trainTarget,test,testTarget,last=None):
    if last!=None:
        originTrain=copy.deepcopy(train)
        originTest=copy.deepcopy(test)
        originTarg=copy.deepcopy(trainTarget)
        originTestTarg=copy.deepcopy(testTarget)
        for m in tqdm(last, position=0, leave=True):
            temp=computeLast(train,m)
            trainTarget-=temp.ravel()
            temp=computeLast(test,m)
            testTarget-=temp.ravel()
            gc.collect()
        guessTe=originTestTarg-testTarget
        del temp
    count=len(train[0])
    print(f"features: {count}")
    model,model2=buildModel(count,probabilistic=(last==None))
    BS=64
    if last!=None:
        loss=tf.keras.losses.MeanSquaredError(name="mse")
        metrics=[tf.keras.metrics.MeanAbsoluteError("mae")]
    else:
        loss=create_weighted_binary_crossentropy(1,1.1)
        metrics=["accuracy",tf.keras.metrics.MeanAbsoluteError("mae")]
    model.compile(loss=loss, metrics=metrics,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))
    model.fit(train,trainTarget,epochs=1,batch_size=BS)
    if last!=None:
        del train
        del trainTarget
        del originTrain
        del originTarg
    gc.collect()
    print("prediction....")
    prediction=model.predict(test)
    if last!=None:
        prediction=guessTe+prediction.ravel()
        prediction=np.clip(prediction,0,1)
        calculate_metrics(prediction,originTestTarg)
    else:
        calculate_metrics(prediction,testTarget)
    return (model,model2)

##Chunk wise download and work

In [ ]:
import os 
from tqdm import tqdm
import json

gc.collect()
modelComp=[]
modelParz=[]
index=0
with open("training_urls.txt") as doc:
    for line in doc:#tqdm(doc, position=0, leave=True):
        #skip lzo.index files (odd row number)
        if index%2==1:
            index+=1
            continue
        os.system(f"aria2c '{line}'")
        partName=str(index//2)
        print(partName)
        partName="0"*(5-len(partName))+partName
        
        os.system(f'lzop -x part-{partName}.lzo')
        os.system(f'mv part-{partName} train.csv')
        os.system(f"rm part-{partName}.lzo")
        train,trainTarget,test,testTarget=computeTrainTest()
        if len(modelComp)==0:
            model1,model2=trainModel(train,trainTarget,test,testTarget)
        else:
            model1,model2=trainModel(train,trainTarget,test,testTarget,last=modelComp)
        modelComp.append(model1)
        modelParz.append(model2)
        os.system("rm train.csv")
        os.system(f"rm part-{partName}.lzo.aria2")
        os.system(f"rm part-{partName}.lzo.index")
        index+=1
        gc.collect()
os.system("mv ./temp.txt ./dictionary.txt")

0


KeyboardInterrupt: ignored

#Code for modeling prediction error giving different weights

##NN model

In [ ]:
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from math import ceil
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.optimizers import RMSprop
from tensorflow.keras import regularizers

In [ ]:
import tensorflow.keras.backend as K
def create_weighted_binary_crossentropy(zero_weight, one_weight):
    def weighted_binary_crossentropy(y_true, y_pred):
        b_ce = K.binary_crossentropy(y_true, y_pred)
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce
        return K.mean(weighted_b_ce)
    return weighted_binary_crossentropy

In [ ]:
def buildModel(inputSize,probabilistic=False):
    model = Sequential()
    size=64
    reduction_factor=2
    shape=(inputSize,)
    depth=2
    dropout=0.4
    BS=16
    kernel_resularizer_norm=1e-5
    for i in range(depth):
        model.add(Dense(size,  input_shape=shape,kernel_regularizer=regularizers.l2(kernel_resularizer_norm)))
        model.add(Activation("relu"))
        model.add(Dropout(dropout))
        size=size//reduction_factor
    model.add(Dense(1,kernel_regularizer=regularizers.l2(1e-4)))

    if (probabilistic):
        model.add(Activation("sigmoid"))
    else:
        pass
        #model.add(Activation("tanh"))
    model2=keras.Model(inputs=[model.inputs],outputs=model.layers[-3].output)

    return (model,model2)

In [ ]:
import copy
def computeLast(arr,last):
    return last.predict(arr)

from tqdm import tqdm
def trainModel(train,trainTarget,test,testTarget,last=None,verbose=False):
    i=0
    alpha=0.75
    lr=0.0003
    if last!=None:
        lr=0.001
        originTrain=copy.deepcopy(train)
        originTest=copy.deepcopy(test)
        originTarg=copy.deepcopy(trainTarget)
        originTestTarg=copy.deepcopy(testTarget)
        
        for m in tqdm(last, position=0, leave=True):
            temp=computeLast(train,m)
            trainTarget-=np.power(alpha,i)*temp.ravel()
            temp=computeLast(test,m)
            testTarget-=np.power(alpha,i)*temp.ravel()
            gc.collect()
            i+=1
        guessTe=originTestTarg-testTarget
        del temp
    count=len(train[0])
    print(f"features: {count}")
    model,model2=buildModel(count,probabilistic=(last==None))
    BS=64
    if last!=None:
        loss=tf.keras.losses.MeanSquaredError(name="mse")
        metrics=[tf.keras.metrics.MeanAbsoluteError("mae")]
    else:
        loss=create_weighted_binary_crossentropy(1,1)
        metrics=["accuracy",tf.keras.metrics.MeanAbsoluteError("mae")]
    model.compile(loss=loss, metrics=metrics,optimizer=tf.keras.optimizers.Adam(learning_rate=lr))
    model.fit(train,trainTarget,epochs=3,batch_size=BS)
    if last!=None:
        del train
        del trainTarget
        del originTrain
        del originTarg
    gc.collect()
    print("prediction....")
    prediction=model.predict(test)
    if last!=None:
        prediction=guessTe+np.power(alpha,i)*prediction.ravel()
        prediction=np.clip(prediction,0,1)
        if verbose:
            for el in prediction:
                if el<0 or el>1:
                    print(el)
        calculate_metrics(prediction,originTestTarg)
    else:
        calculate_metrics(prediction,testTarget)
    return (model,model2)

##Chunk wise download and work

In [ ]:
import os 
from tqdm import tqdm
import json

gc.collect()
modelComp=[]
modelParz=[]
with open("training_urls.txt") as doc:
    index=0
    for line in doc:#tqdm(doc, position=0, leave=True):
        #skip lzo.index files (odd row number)
        if index%2==1:
            index+=1
            continue
        os.system(f"aria2c '{line}'")
        partName=str(index//2)
        print(partName)
        partName="0"*(5-len(partName))+partName
        
        os.system(f'lzop -x part-{partName}.lzo')
        os.system(f'mv part-{partName} train.csv')
        os.system(f"rm part-{partName}.lzo")
        train,trainTarget,test,testTarget=computeTrainTest()
        if len(modelComp)==0:
            model1,model2=trainModel(train,trainTarget,test,testTarget)
        else:
            model1,model2=trainModel(train,trainTarget,test,testTarget,last=modelComp)
        modelComp.append(model1)
        modelParz.append(model2)
        os.system("rm train.csv")
        os.system(f"rm part-{partName}.lzo.aria2")
        os.system(f"rm part-{partName}.lzo.index")
        index+=1
        gc.collect()
os.system("mv ./temp.txt ./dictionary.txt")

In [ ]:
summary

In [ ]:
train[0]

In [ ]:
train,trainTarget,test,testTarget=computeTrainTest()

In [ ]:
originTrain=copy.deepcopy(train)
originTest=copy.deepcopy(test)
originTarg=copy.deepcopy(trainTarget)
originTestTarg=copy.deepcopy(testTarget)
i=0
alpha=0.75        
for m in tqdm(modelComp[:3], position=0, leave=True):
    temp=computeLast(train,m)
    trainTarget-=np.power(alpha,i)*temp.ravel()
    temp=computeLast(test,m)
    testTarget-=np.power(alpha,i)*temp.ravel()
    gc.collect()
    i+=1
guessTe=originTestTarg-testTarget

In [ ]:
guessTr=originTarg-trainTarget

In [ ]:
calculate_metrics(guessTr,originTarg)

rce=7.2338845972432
AP=0.5553876611554468


In [ ]:
i=0
for model in modelComp[:3]:
    model.save(f"baseNet-{targetCol}-{i}-{alpha}.h5")
    i+=1

In [ ]:
summary

Summary:


    {'engaged_with_user_follower_count': {'mean': 842019.06, 'std': 4673210.5},
    'engaged_with_user_following_count': {'mean': 3866.4478,
    'std': 34336.34765625},
    'engaged_with_user_is_verified': {'mean': 0.2052318,
    'std': 0.39793893694877625},
    'engagee_follows_engager': {'mean': 0.40177855, 'std': 0.49429482221603394},
    'engaging_user_follower_count': {'mean': 816.23035, 'std': 10675.9775390625},
    'engaging_user_following_count': {'mean': 716.2875,
    'std': 1651.0892333984375},
    'engaging_user_is_verified': {'mean': 0.0018827069,
    'std': 0.04330842196941376},
    'gifs': {'mean': 0.012480928, 'std': 0.11041624844074249},
    'hashtags': {'mean': 0.41888434, 'std': 1.146403431892395},
    'photos': {'mean': 0.47115853, 'std': 0.8963454961776733},
    'present_domains': {'mean': 0.1264995, 'std': 0.36130860447883606},
    'present_links': {'mean': 0.1264995, 'std': 0.36130860447883606},
    'present_media': {'mean': 0.5532317, 'std': 0.8825932741165161},
    'text_tokens': {'mean': 46.921314, 'std': 29.6999454498291},
    'tweet_type_Quote': {'mean': 0.08192547, 'std': 0.2696485221385956},
    'tweet_type_Retweet': {'mean': 0.32451233, 'std': 0.468744695186615},
    'tweet_type_TopLevel': {'mean': 0.5935622, 'std': 0.49659907817840576},
    'videos': {'mean': 0.069592245, 'std': 0.25196176767349243}}

In [ ]:
import json
with open('summary.json', 'w') as fp:
    json.dump(summary, fp)